In [ ]:
import sys, os
# Need to add parent folder to SYS PATH in order to import packages.
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from buyer_rating_report import NamedExposure
from utils import FORBIDDEN_STATUSES, CORPORATE_COLOR

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
named_exp = NamedExposure()

file_2020_dec = "INWARDS_RE_NAMED Exposure_20201231_BO_v3 (2).txt"
file_2021_apr = "INWARDS_RE_NAMED Exposure_20210430_BO_(testing 18_POT)_v3_MAEDA_removed.txt"
file_2021_may = "INWARDS_RE_NAMED Exposure_20210531_BO_(testing 19_POT)_v1.txt"
file_2021_jun = "INWARDS_RE_NAMED Exposure_20210630_BO_(testing 19_POT)_v1.txt"
file_2021_jul = "INWARDS_RE_NAMED Exposure_20210731_BO_(testing 19_POT)_v1 - sent.txt"
file_2021_sep = "INWARDS_RE_NAMED Exposure_20210930_BO_(testing 19_POT)_v1_sent.txt"
file_2021_oct = "INWARDS_RE_NAMED Exposure_20211031_BO_(testing 21_POT_adjusted)_v1.txt"
file_2021_nov = "INWARDS_RE_NAMED Exposure_20211130_BO_(testing 21_POT_adjusted)_v1.txt"
file_2021_dec = "INWARDS_RE_NAMED Exposure_20211231_BO_(testing 22_POT_adjusted)_v2.txt"
file_2022_jan = "INWARDS_RE_NAMED Exposure_20220131_BO_(testing 22_POT_adjusted)_Jan 22 FX rates_lapsed only.txt"


# Load data into Class
named_exp.import_named_data(file_2020_dec)
named_exp.import_named_data(file_2021_apr)
named_exp.import_named_data(file_2021_may)
named_exp.import_named_data(file_2021_jun)
named_exp.import_named_data(file_2021_jul)
named_exp.import_named_data(file_2021_sep)
named_exp.import_named_data(file_2021_oct)
named_exp.import_named_data(file_2021_nov)
named_exp.import_named_data(file_2021_dec)
named_exp.import_named_data(file_2022_jan)

In [ ]:
#named_exp.compare_dates(to_excel=True, open_file=True)

In [ ]:
from buyer_report_output import BuyerReportHTML
html_rep = BuyerReportHTML(named_exposure=named_exp)
html_rep.to_html()

In [ ]:
df = named_exp.get_portfolio_rating_movement()

fig = go.Figure()

hovertemplate = "<b>%{x}</b><br>"
hovertemplate += "<b>Rating:</b> %{y:.1f}<br>"
hovertemplate += "<b>Change from prev. month:</b> %{customdata[1]:.1%}<br>"

hovertemplate += "<b>Change from {}: ".format(
    df.index[0].strftime('%Y-%b'))
hovertemplate += "</b>%{customdata[2]:.1%}<br>"

hovertemplate += "<extra></extra>"


fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['Rating'],
        customdata=df.fillna(0),
        mode='lines+markers',
        name='lines',
        #hovertemplate='Delta from oldest data: %{customdata[1]:.1%}'
        hovertemplate=hovertemplate
    )
)

# hide and lock down axes
fig.update_xaxes(visible=False, fixedrange=True)
fig.update_yaxes(visible=False, fixedrange=True)
#fig.update_layout(annotations=[], overwrite=True)
fig.update_layout(
    showlegend=False,
    plot_bgcolor="white",
    margin=dict(t=10,l=10,b=10,r=10)
)

fig.show()

In [ ]:
df = html_rep.named_exposure.get_data_for_date('31/12/2020')

In [ ]:
df

In [ ]:
groupby='CEDANT_NAME'
df = named_exp.get_movements(groupby=groupby)

hovertemplate = '<b style="font-size: 1rem">%{label}</b><br>'
hovertemplate += '<b>TPE_EUR-2022-Jan:</b> %{value:,.0f}<br>'
hovertemplate += '<b>RATING-2022-Jan:</b> %{customdata[0]:.1f}<br>'
hovertemplate += '<b>Rating-Diff %:</b> %{color:.2%}<extra></extra>'

fig = px.treemap(
    df,
    path=[px.Constant('Portfolio'), 'CEDANT_NAME'],
    values='TPE_EUR-2022-Jan',
    color='Rating-Diff %',
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
    hover_data=['RATING-2022-Jan'],
    title='Treemap with 2022-Jan information'
)

fig.update_layout(coloraxis_colorbar=dict(
    title="Population",
    tickvals=[df['Rating-Diff %'].min(), 0, df['Rating-Diff %'].max()],
    ticktext=["{:.0%}".format(df['Rating-Diff %'].min()), "0%", "{:.0%}".format(df['Rating-Diff %'].max())],
))

fig.update_traces(hovertemplate=hovertemplate)

In [ ]:
fig.to_dict()

In [ ]:
import plotly.graph_objs as go

groupby='CEDANT_NAME'
df = named_exp.get_movements(groupby=groupby)
df_top = df.nlargest(10, 'Rating-Diff %')
df_bottom = df.nsmallest(10, 'Rating-Diff %').sort_values('Rating-Diff %', ascending=False)

sufx = ['-2021-Dec', '-2022-Jan']

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add old data
fig.add_trace(
    go.Bar(name=sufx[0][1:],
            x=df_top[groupby],
            y=df_top[f'RATING{sufx[0]}'],
            hovertemplate="%{x}<br>Rating: %{y:.1f}",
            marker_color=CORPORATE_COLOR[0])
)

fig.add_trace(
    go.Bar(name=sufx[1][1:],
            x=df_top[groupby],
            y=df_top[f'RATING{sufx[1]}'],
            hovertemplate="%{x}<br>Rating: %{y:.1f}",
            marker_color=CORPORATE_COLOR[1])
)

fig.add_trace(go.Scatter(x=df_top[groupby],
                            y=df_top['Rating-Diff %'],
                            name="MoM % Change",
                            hovertemplate="QoQ : %{y:+.1%}",
                            marker_color=CORPORATE_COLOR[7]),
                secondary_y=True)

fig.update_yaxes(secondary_y=True, 
                    showgrid=False,
                    tickformat='.0%')

#by_plot_title = self.PLOT_LABELS[groupby]

updatemenus = [
    dict(
        type="buttons",
        direction="right",
        buttons=list([
            dict(
                args=[
                    {'y': [df_top[f'RATING{sufx[0]}'],
                           df_top[f'RATING{sufx[1]}'],
                           df_top['Rating-Diff %']],
                     'x': [df_top['CEDANT_NAME'],
                           df_top['CEDANT_NAME'],
                           df_top['CEDANT_NAME']]},
                    {'title.text': 'Top 10 Largest Rating Deterioration by Cedant'}
                ],
                label="Deterioration",
                # relayout is used to change axis from Log to Scatter
                # and vice-versa
                method="update"
            ),
            dict(
                args=[
                    {'y': [df_bottom[f'RATING{sufx[0]}'],
                             df_bottom[f'RATING{sufx[1]}'],
                             df_bottom['Rating-Diff %']],
                    'x': [df_bottom['CEDANT_NAME'],
                          df_bottom['CEDANT_NAME'],
                          df_bottom['CEDANT_NAME']]},
                    {'title.text': 'Top 10 Largest Rating Improvement by Cedant'},
                ],
                label="Improvement",
                method="update"
            )
        ]),
        pad={"l": 10, "t": 10},
        showactive=True,
        # x=1 to position buttons at right-hand side
        x=1,
        xanchor="right",
        # y=1.15 to position buttons at the top
                y=1.15,
                yanchor="top"
    ),
]

fig.update_layout(barmode='group', updatemenus=updatemenus)

#print(fig)

#print(fig)#['data'][0]['x']


# Different option for Broad Rating Band and Weighted Av. Rating